## タスク 1: モデルをトレーニングする

機械学習 (ML) モデルを作成するプロセスは、データ処理から始まります。データ処理が完了したら、モデルをトレーニングする ML アルゴリズムを選択します。モデルトレーニングの目標は、将来のデータで予測を行うために使用できるモデルを作成することです。処理されたデータにはターゲットが含まれている必要がありますが、将来のデータにはターゲットが含まれていません (ラベルが付いていません)。このアルゴリズムは、入力データ属性をターゲットにマッピングするトレーニングデータ内のパターンを検出します。その後、このアルゴリズムはこれらのパターンをキャプチャする ML モデルを出力します。モデルがあれば、ターゲット値を含まない新しいデータでの予測を実行できます。

例えば、メールがスパムかどうかを予測するように ML モデルをトレーニングする場合、ターゲットがわかっているメールを含むトレーニングデータをモデルに提供します (この場合は、メールがスパムかどうかを示すラベル)。アルゴリズムでは、このデータを使用して、メールがスパムかどうかを予測するモデルが作成されます。このモデルを使用して、将来のメールラベルを予測できます。

このタスクでは、誰かの所持金が 50,000 USD 未満かどうかを予測します。このモデルは、誰かの所持金が 50,000 USD 未満かどうかを可能な限り正確に予測するために最適化されるようトレーニングされています。モデルトレーニングでは、トレーニングに使用するアルゴリズムの種類などの設定が必要です。このタスクでは、XGBoost (eXtreme Gradient Boosting) アルゴリズムを使用します。モデルのトレーニングでは、ハイパーパラメータの設定も必要です。ハイパーパラメータは、トレーニングジョブプロセスを制御するパラメータです。トレーニングジョブのさまざまなステップを変更するように調整できます。適切なハイパーパラメータセットを選択することは、モデルのパフォーマンスと精度の観点から重要です。モデルをトレーニングしたら、モデルを評価し、モデルアーティファクトを表示します。

### タスク 1.1: 環境を設定する

モデルのトレーニングを開始する前に、必要な依存関係をすべてインストールします。

In [ ]:
#Install matplotlib and restart kernel
%pip install matplotlib
%pip uninstall bokeh -y
%pip install bokeh==2.4.2
%reset -f

# Install dependencies
import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from sagemaker.debugger import Rule, rule_configs
from IPython.display import FileLink, FileLinks
from sagemaker import image_uris
from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime
from pathlib import Path

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')

注意: 出力に「エラー: 現在、pip の依存関係リゾルバーではインストールされている一部のパッケージが考慮されていません」というエラーが表示される場合があります。パッケージ依存関係エラーは無視して、次のステップに進んでかまいません。

次に、データセットをインポートします。この演習では、Amazon SageMaker Data Wrangler から Amazon Simple Storage Service (Amazon S3) バケットにデータファイルがエクスポートされていると仮定します。データセットをトレーニングデータセット (70%)、検証データセット (20%)、テストデータセット (10%) に分割します。トレーニングデータセットと検証データセットはトレーニング中に使用します。テストデータセットはデプロイ後のモデル評価で使用します。

Amazon SageMaker を使用してトレーニングするには、データセットが libSVM 形式と CSV 形式のどちらかである必要があります。このラボではトレーニングに CSV 形式を使用します。

前のラボで作成したデータセットファイルを表示するには、以下の手順に従います。

1.左側のメニューバーからバケットアイコンを選択します。

1.バケットのリストで、名前に **labdatabucket** が含まれている Amazon S3 バケットを選択します。

.csv ファイルを開くと、SageMaker Studio で新しいタブが開きます。以下のオプションの 1 つを実行して、手順を進めます。
- **オプション 1:** タブを並べて表示します。メインの SageMaker Studio ウィンドウから分割画面ビューを作成するには、[**llab_2.ipynb**] タブを横にドラッグするか、[**lab_2.ipynb**] タブを選択し、ツールバーから [**ファイル**] および [**New View for Notebook**] を選択します。これで、手順を表示しながら .csv ファイルを調査できるようになります。
- **オプション 2:** SageMaker Studio のタブを切り替えながら、手順を進めます。.csv ファイルの調査を終了する場合は、[**lab_2.ipynb**] タブを選択してノートブックに戻ります。

1.[**scripts**] フォルダを選択 (ダブルクリック) し、[**data**] フォルダを選択 (ダブルクリック) してから、[**train**] フォルダを選択 (ダブルクリック) し、[**adult_data_processed_train.csv**] ファイルを選択 (ダブルクリック) してその内容を表示します。

1.左側のペインで、<i aria-hidden="true" class="fas fa-folder" style="color:white"></i>[**/ ... /data/train/**] パンくずリストリンクから [**data**] を選択します。

1.[**validation**] フォルダを選択 (ダブルクリック) し、[**adult_data_processed_validation.csv**] ファイルを選択 (ダブルクリック) してその内容を表示します。

データセットファイルを表示しました。次に、トレーニングジョブで入力として使用されるトレーニングパスと検証パスを設定します。

In [ ]:
# Import the datasets
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)
prefix = 'scripts/data'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Configure the training paths
train_path = f"s3://{bucket}/{prefix}/train/adult_data_processed_train.csv"
validation_path = f"s3://{bucket}/{prefix}/validation/adult_data_processed_validation.csv"

# Set up the TrainingInput objects
train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

# Print the training and validation paths
print(f'Training path: {train_path}')
print(f'Validation path: {validation_path}')

# Set the container, name, and tags
create_date = strftime("%m%d%H%M")
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')
run_name = 'lab-2-run-{}'.format(create_date)

### タスク 1.2: 推定器オブジェクトを設定する

推定器は、SageMaker トレーニングの高レベルインターフェイスです。推定器オブジェクトを作成するには、AWS Identity and Access Management (IAM) ロール、コンピューティングインスタンスの数とタイプ、Amazon S3 出力パスなどの必須パラメータを指定します。このラボでは、SageMaker の汎用推定器に XGBoost 組み込みアルゴリズムを使用します。XGBoost は、勾配ブーストツリーアルゴリズムの一般的で効率的なオープンソースの実装です。勾配ブースティングは教師あり学習アルゴリズムで、より単純で弱いモデルのセットからの推定のアンサンブルを組み合わせることで、ターゲット変数の正確な予測を試行します。XGBoost アルゴリズムは、さまざまなデータタイプ、関連性、ディストリビューション、およびファインチューニングできるさまざまなハイパーパラメータの処理に適しています。XGBoost は、回帰、分類 (バイナリおよびマルチクラス)、ランク付けの問題に使用できます。この場合、XGBoost を使用して分類の問題 (誰かの収入が 50,000 USD 未満かどうか) を解決します。

このラボでは、*sagemaker.Estimator.Estimator* クラスを使用して XGBoost 推定器を作成します。次のサンプルコードでは、XGBoost 推定器の名前は *xgb_model* です。SageMaker 推定器を構築するには、以下のパラメータを指定します。

- **image_uri**: トレーニングコンテナイメージの URI。この例では、SageMaker XGBoost トレーニングコンテナの URI は *image_uris.retrieve* を使用して指定されます。
- **role**: SageMaker がユーザーに代わってタスク (トレーニング結果の読み取り、Amazon S3 からのモデルアーティファクトの呼び出し、Amazon S3 へのトレーニング結果の書き込みなど) を実行するために使用する IAM ロール。
- **instance_count and instance_type**: モデルトレーニングに使用する Amazon EC2 ML コンピューティングインスタンスのタイプと数。このラボでは、4 つの CPU、16 GB のメモリ、Amazon Elastic Block Store (Amazon EBS) ストレージを備え、高いネットワークパフォーマンスを発揮する 1 つの ml.m5.xlarge インスタンスを使用します。
- **output_path**: SageMaker がモデルアーティファクトとトレーニング結果を保存する S3 バケットへのパス。
- **sagemaker_session**: トレーニングジョブで使用される SageMaker API 操作やその他の AWS サービスとのインタラクションを管理するセッションオブジェクト。
- **rules**: Amazon SageMaker Debugger の組み込みルールのリスト。この例では、create_xgboost_report() ルールで、トレーニングの進捗状況と結果に関するインサイトを提供する XGBoost レポートが作成されます。

In [ ]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role, 
    instance_count = 1, 
    instance_type ='ml.m5.xlarge',
    output_path = output_path,
    sagemaker_session = sagemaker_session,
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report())
    ]
)

### タスク 1.3: ハイパーパラメータを設定する

ハイパーパラメータは、モデルの構造、関数、パフォーマンスを直接制御します。ハイパーパラメータのチューニングにより、データサイエンティストはモデルのパフォーマンスをファインチューニングして最適な結果を得ることができます。このプロセスは機械学習の重要な部分であり、適切なハイパーパラメータ値を選択することが成功に不可欠です。

XGBoost アルゴリズムのハイパーパラメータを設定するには、推定器の *set_hyperparameters* メソッドを呼び出します。

XGBoost ハイパーパラメータの詳細については、[XGBoost ハイパーパラメータ](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)を参照してください。

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    verbosity = 0,
    objective = 'binary:logistic',
    num_round = 800
)

### タスク 1.4: SageMaker トレーニングジョブを実行する

推定器オブジェクトとハイパーパラメータの設定が完了し、モデルのトレーニングを開始する準備が整いました。fit() メソッドは、トレーニングスクリプトを起動します。モデルトレーニングを開始するには、トレーニングデータセットと検証データセットを使用して推定器の fit() メソッドを呼び出します。「wait=True」に設定すると、fit() メソッドは進行状況ログを表示し、トレーニングが完了するまで待機します。

In [ ]:
xgb_model.fit(
    {
        "train": train_input,
        "validation": validation_input
    },
    wait=True
)

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i>**正常な出力:** 推定器とハイパーパラメータの設定が適切で、トレーニングジョブが正しく開始されていれば、出力は次のようになるはずです。

```plain
************************
**** EXAMPLE OUTPUT ****
************************

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-09-20-09-56-628
2023-08-09 20:09:56 Starting - Starting the training job...
2023-08-09 20:10:19 Starting - Preparing the instances for trainingCreateXgboostReport: InProgress
......
2023-08-09 20:11:21 Downloading - Downloading input data...
2023-08-09 20:11:55 Training - Downloading the training image...
2023-08-09 20:12:20 Training - Training image download completed.Training in progress....
2023-08-09 20:12:56 Uploading - Uploading generated training model...
2023-08-09 20:13:20 Completed - Training job completed
..Training seconds: 107
Billable seconds: 107
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **注意:** このトレーニングの所要時間は約 3～4 分です。

### タスク 1.5: モデルを評価する

トレーニングジョブが完了したら、SageMaker Debugger で生成された XGBoost トレーニングレポートをダウンロードできます。XGBoostトレーニングレポートでは、反復に関する損失関数、特徴量の重要度、混同行列、正解率曲線、トレーニングのその他の統計結果など、トレーニングの進行状況と結果に関するインサイトが得られます。

SageMaker XGBoost トレーニングジョブでは、デバッガーの「CreateXgboostReport」ルールを使用して、トレーニングの進捗状況と結果に関する包括的なトレーニングレポートを受け取ることができます。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意:** レポートが完成するまでの所要時間は 5～10 分です。次のコードブロックでは、最終レポートが存在するかどうかが確認されます。まだ存在しない場合は 30 秒間待機し、新しいファイルを出力フォルダからローカルパスにコピーして、レポートが使用可能になるまで確認を繰り返します。

In [ ]:
%%capture
the_file = Path("CreateXgboostReport/xgboost_report.ipynb")
notexists = True
while notexists:
    if the_file.is_file():
        notexists = False
    else:
        time.sleep(30)
        rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.job_name + "/rule-output"
        ! aws s3 ls {rule_output_path} --recursive
        ! aws s3 cp {rule_output_path} ./ --recursive

次のセルの出力内のリンクをクリックすると、SageMaker Studio で新しいタブが開きます。以下のオプションの 1 つを実行して、手順を進めます。
- **オプション 1:** タブを並べて表示します。メインの SageMaker Studio ウィンドウから分割画面ビューを作成するには、[**llab_2.ipynb**] タブを横にドラッグするか、[**lab_2.ipynb**] タブを選択し、ツールバーから [**ファイル**] および [**New View for Notebook**] を選択します。これで、手順を表示しながら XGBoost レポートを調査できるようになります。
- **オプション 2:** SageMaker Studio のタブを切り替えながら、手順を進めます。XGBoost レポートの調査を終了する場合は、[**lab_2.ipynb**] タブをクリックしてノートブックに戻ります。

In [ ]:
display("Click link below to view the XGBoost Training notebook", FileLink("CreateXgboostReport/xgboost_report.ipynb"))

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **注意:** このコードを実行すると、次の出力が表示されるはずです。**'Click link below to view the XGBoost Training notebook' <span style="ssb_sm_blue">CreateXgboostReport/xgboost_report.ipynb</span>**

ノートブックを新しいタブで開くには、リンクを選択します。ノートブックが開いたら、[**Set up notebook environment**] ウィンドウで、以下を設定します。

- [**Image**] には、[**Data Science 3.0**] を選択します。
- [**Kernel**] には、[**Python 3**] を選択します。
- [**選択**] をクリックします。

[**xgboost_report.ipynb**] タブの上部で、<i aria-hidden="true" class="fas fa-forward"></i>[**Restart Kernel and Run All Cells**] ボタンをクリックします。[**Restart Kernel?**] が表示されたら、[**Restart**] をクリックします。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意:** すべてのセルを実行するまでの所要時間は約 2～3 分です。

すべてのセルの実行が終了したら、[**Confusion Matrix**] が表示されるまで下にスクロールします。混同行列は、観測値の予測クラスと実際のクラスを比較することで、各クラスの正しい予測と誤った予測の数を表に示します。図を見ると、**真陽性 (TP)**、**真陰性 (TN)**、**偽陽性 (FP)**、**偽陰性 (FN)** の値があります。

- **真陽性:** 実際の分類が陽性で、予測分類が陽性 (1,1) の場合、陽性サンプルが分類器によって正しく識別されたため、**真陽性 (TP)** 結果と呼ばれます。
- **偽陰性:** 実際の分類が陽性で、予測分類が陰性 (1,0) の場合、陽性サンプルが分類器によって陰性と誤って識別されたため、**偽陰性 (FN)** 結果と呼ばれます。
- **偽陽性:** 実際の分類が陰性で、予測分類が陽性 (0,1) の場合、陰性サンプルが分類器によって陽性と誤って識別されたため、**偽陽性 (FP)** 結果と呼ばれます。
- **真陰性**: 実際の分類が陰性で、予測分類が陰性 (0,0) の場合、陰性サンプルが分類器によって正しく識別されたため、**真陰性 (TN)** 結果と呼ばれます。

次に、[**Evaluation of the Confusion Matrix**] まで下にスクロールし、[**Classification report**] を詳しく見て、各クラスの適合率、再現率、F1 スコアの概要を理解します。

- **適合率**: 陽性と予測された総数に対する、陽性と予測された実際の陽性数の割合を測定します。範囲は 0～1 で、値が大きいほど正解率が高いことを示します。適合率は、モデルで関連性があると示されたデータポイントと、実際に関連性があったデータポイントの比率を表します。適合率は、特に FP のコストが高い場合に考慮すべき有用な指標です。
- **再現率/感度/真陽性率 (TPR)**: 実際の陽性のうち、陽性と予測された割合を測定します。こちらも範囲は 0～1 で、値が大きいほど予測正解率が高いことを示します。これは再現率/感度とも呼ばれます。この指標は、データセット内の関連インスタンスをすべて見つける能力を表します。
- **F1 スコア**: 適合率と再現率の調和平均であるターゲットメトリクスを示します。F1 では FP と FN の両方が考慮され、適合率と再現率に同じ重みが与えられます。

政府の支援サービスを資格のある市民に周知するために、収入が 50,000 USD 未満かどうかを予測しようとしています。この場合、F1 スコアが有用な指標です。この指標では、FP (収入が 50,000 USD 以上であるものの 50,000 USD 未満とラベル付けされた人) と FN (収入が 50,000 USD 以下であるものの 50,000 USD 以上とラベル付けされた人) を考慮するためです。適合率と再現率の両方を高くする必要があり、F1 スコアでは両方の指標が考慮されます。次のラボでは、ハイパーパラメータをチューニングしてモデルを最適化し、F1 スコアが向上するかどうかを確認します。

このモデルの**適合率**、**再現率**、**F1 スコア**、**総合正解率**はどのくらいでしょうか。

<i aria-hidden="true" class="far fa-comment" style="color:#008296"></i>**考慮すべき点:** ノートブックに含まれている他のグラフも確認してください。どのような情報が表示されますか。 モデルのトレーニングで、何が役に立つ可能性がありますか。

### タスク 1.6: モデルアーティファクトを表示する

SageMaker ではモデルアーティファクトが S3 バケットに保存されます。モデルアーティファクトの場所を見つけるには、次の手順に従います。

1.左側のメニューバーからバケットアイコンを選択します。

1.バケットのリストで、名前に **labdatabucket** が含まれている Amazon S3 バケットを開きます。

1.**scripts/data/output/ sagemaker-xgboost-.../output** サブフォルダに移動します。

このサブフォルダにモデルアーティファクト **model.tar.gz** が表示されます。これは SageMaker 推定器で fit() メソッドを呼び出して作成したモデルです。

model.tar.gz ファイルを含むモデルアーティファクトを表示しました。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じる。
- 実習セッションに戻り、タスク 2 に進む。